# 6.2 Contact Problems

work in progress ...

In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

rect = MoveTo(0,-0.02).Rectangle(0.04, 0.02).Face()
ball = MoveTo(0,0).Arc(0.005, 90).Rotate(90).Line(0.005).Close().Face()

rect.edges.Min(X).name="sym"
rect.edges.Max(X).name="free"
rect.edges.Min(Y).name="fix"
rect.edges.Max(Y).name="contact1"
ball.edges.Min(X).name="sym"
ball.edges.Max(Y).name="disp"
ball.edges.Max(X-Y).name="contact2"
ball.faces.name="ball"
rect.faces.name="rect"
rect = Glue ([rect, Vertex((0.001,0,0))])
rect.vertices.Max(Y-X).maxh=0.0002
rect.edges.Max(Y-0.1*X).maxh=0.00002
geo = Compound([rect,ball])
DrawGeo (geo)
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.002))
mesh.Curve(4)
Draw(mesh);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.02, -0.0075, 0.0], 'mesh…

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

In [2]:
mesh.GetBoundaries(), mesh.GetMaterials()

(('fix', 'free', 'contact1', 'contact1', 'sym', 'contact2', 'disp', 'sym'),
 ('rect', 'ball'))

In [3]:
E = 2.1e11
nu = 0.2

rho = 7e3
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

I = Id(mesh.dim)
def Pow(a, b):
    return a**b  # exp (log(a)*b)

def C(u):
    # F = Id(2) + Grad(u)
    G = Grad(u)
    F = Id(3) + CoefficientFunction( (G[0,0], G[1,0], 0,   G[1,0], G[1,1], 0,    0, 0, u[0]/(x+1e-16) ) )
    return F.trans*F

def NeoHooke (C):
    return 0.5 * mu * (Trace(C-Id(C.dims[0])) + 2*mu/lam * Det(C)**(-lam/2/mu) - 1)

fes = VectorH1(mesh, order=4, dirichletx="sym", dirichlety="fix|disp")
u,v = fes.TnT()

a = BilinearForm(fes)
a += Variation((x*NeoHooke(C(u))).Compile()*dx)

contact = ContactBoundary(mesh.Boundaries("contact1"), mesh.Boundaries("contact2"))

X = CoefficientFunction((x,y))
cf = (X + u - (X.Other() + u.Other())) * contact.normal

# energy formulation
# contact.AddEnergy(IfPos(cf, 1e14*cf*cf, 0))
# or integrator
contact.AddIntegrator(IfPos(cf, 1e14*cf*(v-v.Other())*contact.normal, 0))

gfu = GridFunction(fes)

disp = -10e-6
gfu.Set((0, disp), definedon=mesh.Materials("ball"))

sceneu = Draw(gfu, deformation=gfu)

C_ = C(gfu)
sigma = NeoHooke(C_).Diff(C_)
scenesigma = Draw(sigma[1,1], mesh, "sigmazz", min=-2e9, max=2e9)


with TaskManager():
    contact.Update(gfu, a, 200, 1e-4)

    NewtonMinimization(a=a, u=gfu, printing=False, inverse="sparsecholesky")
    sceneu.Redraw()
    scenesigma.Redraw()

Draw(sigma[1,1], mesh, "sigmazz", min=-5e9, max=0, deformation=gfu);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

## Dynamic Contact

In [4]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

In [5]:
bowl = MoveTo(0,-0.01).Arc(0.2,70).Rotate(90).Line(0.01).Rotate(90) \
    .Arc(0.19,-140).Rotate(90).Line(0.01).Rotate(90).Arc(0.2,70).Face()
ball1 = Circle((0,0.1),0.01).Face()
ball2 = Circle((0.05,0.1),0.01).Face()
ball3 = Circle((-0.05,0.1),0.01).Face()
balls = Compound([ball1, ball2, ball3])
balls.edges.name = "balls"
geo = Compound([bowl, balls])
bowl.edges.name="contact"
bowl.edges.Min(Y+0.01*X).name="fix"
bowl.edges.Min(Y-0.01*X).name="fix"
DrawGeo(geo);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.0, 0.057508086384061484,…

In [6]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.005))
mesh.Curve(4)
Draw(mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

In [7]:
mesh.GetBoundaries()

('fix', 'contact', 'contact', 'contact', 'fix', 'balls', 'balls', 'balls')

In [12]:
E, nu = 210e6, 0.2
rho = 7e3
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

I = Id(mesh.dim)

def C(u): 
    F = I+Grad(u)
    return F.trans*F
def NeoHooke (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Det(C)**(-lam/2/mu) - 1)

fes = VectorH1(mesh, order=3, dirichlet="fix")
u,v = fes.TnT()

force = CF((0,-9.81*rho))

uold = GridFunction(fes)
unew = GridFunction(fes)
vel = GridFunction(fes)
anew = GridFunction(fes)
aold = GridFunction(fes)

tau = 2e-4

bfmstar = BilinearForm(fes)
bfmstar += Variation( NeoHooke (C(u)).Compile(False)*dx )
bfmstar += Variation( -force*u*dx )
bfmstar += Variation( rho/2* 2/tau**2 * (u-uold-tau*vel-tau**2/4*aold)**2 * dx )

tend = 1

scene = Draw (unew, mesh, "disp", deformation=unew)

t = 0
unew.Set( (0,0) )
vel.Set( (0,0) )
contact = ContactBoundary(mesh.Boundaries("contact|balls"), mesh.Boundaries("contact|balls"))

X = CoefficientFunction((x,y))

if True:
    cf = (X + u-uold - (X.Other() + u.Other() - uold.Other())) * (specialcf.normal(2).Other())
    # cf = (X + u-uold)*specialcf.normal(2) + \
    #    (X.Other() + u.Other() - uold.Other()) * specialcf.normal(2).Other()
    contact.AddEnergy(IfPos(cf, 1e9*cf*cf, 0), deformed=True)
else:
    cf = (X + u - (X.Other() + u.Other())) * contact.normal
    contact.AddEnergy(IfPos(cf, 1e9*cf*cf, 0), deformed=False)


WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

In [13]:
with TaskManager():
    while t < tend:
        print ("time", t)
        t += tau
        uold.vec.data = unew.vec
        aold.vec.data = anew.vec

        contact.Update(uold, bfmstar, 5, 0.01)
        NewtonMinimization (a=bfmstar, u=unew, printing=False, inverse="sparsecholesky")

        anew.vec.data = unew.vec-uold.vec-tau*vel.vec-tau**2/4*aold.vec
        anew.vec.data *= 4/tau**2
        vel.vec.data += 0.5*tau*aold.vec
        vel.vec.data += 0.5*tau*anew.vec

        scene.Redraw()

time 0
time 0.0002
time 0.0004
time 0.0006000000000000001
time 0.0008
time 0.001
time 0.0012000000000000001
time 0.0014000000000000002
time 0.0016000000000000003
time 0.0018000000000000004
time 0.0020000000000000005
time 0.0022000000000000006
time 0.0024000000000000007
time 0.0026000000000000007
time 0.002800000000000001
time 0.003000000000000001
time 0.003200000000000001
time 0.003400000000000001
time 0.003600000000000001
time 0.0038000000000000013
time 0.004000000000000001
time 0.004200000000000001
time 0.0044
time 0.0046
time 0.0048
time 0.004999999999999999
time 0.005199999999999999
time 0.0053999999999999986
time 0.005599999999999998
time 0.005799999999999998
time 0.0059999999999999975
time 0.006199999999999997
time 0.006399999999999997
time 0.0065999999999999965
time 0.006799999999999996
time 0.006999999999999996
time 0.0071999999999999955
time 0.007399999999999995
time 0.007599999999999995
time 0.0077999999999999944
time 0.007999999999999995
time 0.008199999999999995
time 0.0083

time 0.06719999999999987
time 0.06739999999999988
time 0.06759999999999988
time 0.06779999999999989
time 0.0679999999999999
time 0.0681999999999999
time 0.0683999999999999
time 0.06859999999999991
time 0.06879999999999992
time 0.06899999999999992
time 0.06919999999999993
time 0.06939999999999993
time 0.06959999999999994
time 0.06979999999999995
time 0.06999999999999995
time 0.07019999999999996
time 0.07039999999999996
time 0.07059999999999997
time 0.07079999999999997
time 0.07099999999999998
time 0.07119999999999999
time 0.07139999999999999
time 0.0716
time 0.0718
time 0.07200000000000001
time 0.07220000000000001
time 0.07240000000000002
time 0.07260000000000003
time 0.07280000000000003
time 0.07300000000000004
time 0.07320000000000004
time 0.07340000000000005
time 0.07360000000000005
time 0.07380000000000006
time 0.07400000000000007
time 0.07420000000000007
time 0.07440000000000008
time 0.07460000000000008
time 0.07480000000000009
time 0.0750000000000001
time 0.0752000000000001
time 0

time 0.13340000000000177
time 0.13360000000000177
time 0.13380000000000178
time 0.13400000000000178
time 0.1342000000000018
time 0.1344000000000018
time 0.1346000000000018
time 0.1348000000000018
time 0.1350000000000018
time 0.13520000000000182
time 0.13540000000000182
time 0.13560000000000183
time 0.13580000000000184
time 0.13600000000000184
time 0.13620000000000185
time 0.13640000000000185
time 0.13660000000000186
time 0.13680000000000186
time 0.13700000000000187
time 0.13720000000000188
time 0.13740000000000188
time 0.1376000000000019
time 0.1378000000000019
time 0.1380000000000019
time 0.1382000000000019
time 0.1384000000000019
time 0.13860000000000192
time 0.13880000000000192
time 0.13900000000000193
time 0.13920000000000193
time 0.13940000000000194
time 0.13960000000000194
time 0.13980000000000195
time 0.14000000000000196
time 0.14020000000000196
time 0.14040000000000197
time 0.14060000000000197
time 0.14080000000000198
time 0.14100000000000198
time 0.141200000000002
time 0.14140

KeyboardInterrupt: 

## A 3D example

In [14]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

In [15]:
box = Box((0,0,0), (1,1,0.5))
rod = Cylinder((0.5, 0.5, -0.5),Z,0.2,1.5)
rod.faces.name="contact1"
block = box-rod
block.faces.Min(X).name="fix"
rod.faces.name="contact2"
rod.faces.Max(Z).name="force"
rod.faces.Min(Z).name="free"
geo = Compound([block, rod])
DrawGeo (geo);
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.1)).Curve(3)
Draw(mesh);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.5, 0.5, 0.25], 'mesh_rad…

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

In [16]:
mesh.GetBoundaries()

('fix',
 'bc_1',
 'bc_2',
 'bc_3',
 'bc_4',
 'bc_5',
 'contact1',
 'contact2',
 'force',
 'free')

In [28]:
fes = VectorH1(mesh, order=3, dirichlet="fix")
u = fes.TrialFunction()
gfu = GridFunction(fes)

a = BilinearForm(fes)
a += Variation(InnerProduct(Sym(Grad(u)), Sym(Grad(u)))*dx)
a += Variation(1e-8*u*u*dx)   # regularization
a += Variation(-0.01*u[0]*ds("force"))

In [29]:
contact = ContactBoundary(mesh.Boundaries("contact1"), mesh.Boundaries("contact2"))
X = CoefficientFunction((x,y,z))
cf = (X + u - (X.Other() + u.Other())) * specialcf.normal(3)
contact.AddEnergy(IfPos(cf, 1e4*cf*cf, 0))
contact.Update(gfu, a, 10, 4)

In [30]:
with TaskManager(pajetrace=10**9):
    NewtonMinimization(a=a, u=gfu, printing=True, inverse="sparsecholesky");

Newton iteration  0
Energy:  2.88467103341253e-06
err =  0.010554943117905089
Newton iteration  1
Energy:  -4.042924213213675e-05
err =  0.005127025711813756
Newton iteration  2
Energy:  -5.3519529639782634e-05
err =  0.002027429548985573
Newton iteration  3
Energy:  -5.205511799337256e-05
err =  0.0032236945567533054
Newton iteration  4
Energy:  -5.709415391994757e-05
err =  0.0020459749040875404
Newton iteration  5
Energy:  -5.839347526274963e-05
err =  0.0021813083238465636
Newton iteration  6
Energy:  -6.0699800912007175e-05
err =  0.0018553688638011578
Newton iteration  7
Energy:  -6.215252548438982e-05
err =  0.0024247699188785256
Newton iteration  8
Energy:  -6.50199288491952e-05
err =  0.0021286601929742994
Newton iteration  9
Energy:  -6.760045876439849e-05
err =  0.003033708774036154
Newton iteration  10
Energy:  -7.141128864379365e-05
err =  0.0014088736827078995
Newton iteration  11
Energy:  -7.163315353313043e-05
err =  0.001356556588032931
Newton iteration  12
Energy:  -7

In [25]:
Draw (gfu, mesh, deformation=True);
Draw (Sym(Grad(gfu))[0,0], mesh, draw_surf=False);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

WebGuiWidget(value={'ngsolve_version': '6.2.2104-163-g7bf16d002', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

In [27]:
help (contact.Update)

Help on method Update in module ngsolve.comp:

Update(...) method of ngsolve.comp.ContactBoundary instance
    Update(self: ngsolve.comp.ContactBoundary, gf: ngsolve.comp.GridFunction, bf: ngsolve.comp.BilinearForm = None, intorder: int = 4, maxdist: float = 0.0) -> None
    
    
    Update searchtree for gap function.
    If bf is given add specialelements corresponding to
    integrationrules of order 'intorder' on each master
    element to BilinearForm bf.
    `maxdist` is the maximum distance where this function is accurate.
    If `maxdist` == 0. then 2*meshsize is used.

